<a href="https://colab.research.google.com/github/MatteoBettini/Stock-Market-Prediction-2020/blob/main/notebooks/Data%20exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [1]:
import pandas as pd
import numpy as np

In [7]:
def series_to_supervised(data, n_in, dropnan=False):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t)
    cols.append(df.iloc[:][0] - df.shift(1).iloc[:][0])

    names += ['target']
  
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [9]:
dataset_url = 'https://raw.githubusercontent.com/MatteoBettini/Stock-Market-Prediction-2020/main/stock_markets_datasets/Processed_NASDAQ.csv?token=ANHXQQK4VPBE6ABSBCHTF5K74UP3W'
nasdaq_df = pd.read_csv(dataset_url)
# Dataset is now stored in a Pandas Dataframe
nasdaq_df = nasdaq_df.drop(columns=['Date','Name'])
nasdaq_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1984 entries, 0 to 1983
Data columns (total 82 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Close           1984 non-null   float64
 1   Volume          1983 non-null   float64
 2   mom             1983 non-null   float64
 3   mom1            1982 non-null   float64
 4   mom2            1981 non-null   float64
 5   mom3            1980 non-null   float64
 6   ROC_5           1979 non-null   float64
 7   ROC_10          1974 non-null   float64
 8   ROC_15          1969 non-null   float64
 9   ROC_20          1964 non-null   float64
 10  EMA_10          1975 non-null   float64
 11  EMA_20          1965 non-null   float64
 12  EMA_50          1935 non-null   float64
 13  EMA_200         1785 non-null   float64
 14  DTB4WK          1984 non-null   float64
 15  DTB3            1984 non-null   float64
 16  DTB6            1984 non-null   float64
 17  DGS5            1984 non-null   f

In [8]:
data = series_to_supervised(nasdaq_df.values, 40)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1984 entries, 0 to 1983
Columns: 3281 entries, var1(t-40) to target
dtypes: float64(3281)
memory usage: 49.7 MB
